In [5]:
%load_ext autoreload
%autoreload 2

from nirs4all.pipeline.runner import PipelineRunner
from nirs4all.pipeline.config import PipelineConfig
from nirs4all.dataset import SpectroDataset
from sample import config as python_config
from nirs4all.dataset.loader import get_dataset
from nirs4all.controllers.registry import reset_registry
from nirs4all.controllers import *

dataset_py = get_dataset(python_config["dataset"])
print("\n", "="*200, "\nPython Dataset:\n", dataset_py)

print("\n", "="*200, "\nRunning Python Config:\n")
py_config = PipelineConfig(python_config)
# json_config = PipelineConfig("sample.json")
# yaml_config = PipelineConfig("sample.yaml")

runner = PipelineRunner(max_workers=4, continue_on_error=False)
res_dataset, history, pipeline = runner.run(py_config, dataset_py)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
>> Browsing ./sample_data
No train_group file found for ./sample_data.
No test_group file found for ./sample_data.


sys:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
sys:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance



Python Dataset:
 FeatureBlock with 1 sources and 189 samples
Source 0: FeatureSource(shape=(189, 1, 2151), dtype=float32, processing_ids=['raw'], mean=0.46620577573776245, variance=0.1492786705493927)
Index:
shape: (189, 7)
┌─────┬────────┬────────┬───────────┬───────┬────────┬────────────┐
│ row ┆ sample ┆ origin ┆ partition ┆ group ┆ branch ┆ processing │
│ --- ┆ ---    ┆ ---    ┆ ---       ┆ ---   ┆ ---    ┆ ---        │
│ i32 ┆ i32    ┆ i32    ┆ cat       ┆ i8    ┆ i8     ┆ cat        │
╞═════╪════════╪════════╪═══════════╪═══════╪════════╪════════════╡
│ 0   ┆ 0      ┆ null   ┆ train     ┆ 0     ┆ 0      ┆ raw        │
│ 1   ┆ 1      ┆ null   ┆ train     ┆ 0     ┆ 0      ┆ raw        │
│ 2   ┆ 2      ┆ null   ┆ train     ┆ 0     ┆ 0      ┆ raw        │
│ 3   ┆ 3      ┆ null   ┆ train     ┆ 0     ┆ 0      ┆ raw        │
│ 4   ┆ 4      ┆ null   ┆ train     ┆ 0     ┆ 0      ┆ raw        │
│ …   ┆ …      ┆ …      ┆ …         ┆ …     ┆ …      ┆ …          │
│ 184 ┆ 184    ┆ null   ┆ t

Traceback (most recent call last):
  File "D:\Workspace\ML\NIRS\nirs4all\nirs4all\pipeline\runner.py", line 138, in _run_step
    return self._execute_controller(controller, step, operator, dataset, context)
  File "D:\Workspace\ML\NIRS\nirs4all\nirs4all\pipeline\runner.py", line 195, in _execute_controller
    return controller.execute(step, operator, dataset, context, self, source)
  File "D:\Workspace\ML\NIRS\nirs4all\nirs4all\controllers\sklearn\op_transformermixin.py", line 50, in execute
    transformed_data = operator.transform(transformed_data)
  File "d:\Workspace\ML\NIRS\nirs4all\.venv\lib\site-packages\sklearn\utils\_set_output.py", line 319, in wrapped
    data_to_wrap = f(self, X, *args, **kwargs)
  File "D:\Workspace\ML\NIRS\nirs4all\nirs4all\operators\augmentation\abc_augmenter.py", line 113, in transform
    return self.augment(X, self.apply_on)
  File "D:\Workspace\ML\NIRS\nirs4all\nirs4all\operators\augmentation\random.py", line 99, in augment
    increment = np.array

RuntimeError: Pipeline step failed: Pipeline step failed: zero-size array to reduction operation maximum which has no identity

In [ ]:
print(dataset_py.features.index)

SyntaxError: incomplete input (307681973.py, line 1)